In [1]:
import numpy as np 
import pandas as pd 
import pymysql
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import preprocessing
%matplotlib notebook
import matplotlib.pyplot as plt, mpld3
from matplotlib import rcParams
from matplotlib.cm import rainbow
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import State, Input, Output
import dash_bootstrap_components as dbc
import dash_auth
from flask import Flask, redirect
from pathlib import Path
import dash_table
import plotly.graph_objs as go
engine = create_engine('mysql+pymysql://root:@localhost/student')
mycursor= engine.connect()
colors = ['crimson','lightslategray','yellow','blue','red','pink','green'] 
def importdata(): 
    balance_data = pd.read_excel('dataset.xlsx') 
    data_tests=pd.read_excel('testdata.xlsx')
    return balance_data,data_tests
def splitdataset(balance_data): 

    # Separating the target variable 
    X = balance_data.values[:, 2:4] 
    Y = balance_data.values[:, 4]
    #print(X)
    # Splitting the dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(  X, Y, test_size = 0.3, random_state = 100) 
    return X, Y, X_train, X_test, y_train, y_test 
def train_using_gini(X_train, X_test, y_train,y_test): 
    
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, min_samples_leaf=2) 
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 
def tarin_using_entropy(X_train, X_test, y_train): 

    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( criterion = "entropy", random_state = 100, max_depth = 3, min_samples_leaf = 2) 
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy 
def predictionres(data_tests, clf_object): 
    originaldata=data_tests
    # Predicton on test with giniIndex 
    datapred=[]
    #print(data_tests)
    #print(clf_object)
    datay_pred = clf_object.predict(data_tests) 
    #print("Predicted values:") 
    #print(datay_pred)
    datapred=list( datay_pred)
    #print(datapred)
    return datapred

def generate_graph(dataframe,i):
    
    return  html.Div(
                [
                    html.H3(i),
                    dcc.Graph(
                        figure={
                            "data": [
                               go.Bar(
                                    x=dataframe["GRADE"].unique(),
                                    y=dataframe["GRADE"].value_counts(),
                                    marker_color=colors
                                )]})
                ])
def generate_piechart(dataframe,f):
    w=len(dataframe)
    lis=[]
    lis.append(f)
    passs=w-f
    lis.append(passs)
    #print(lis)
    return  html.Div(
                [
                    html.H3('passs% vs fail%'),
                    dcc.Graph(
                        figure={
                            "data": [
                               go.Pie(
                                    values=lis,
                                    labels=['fail%','pass%']
                                )]})
                ])
    
    
def generate_table(dataframe):
    max_rows=len(dataframe)
    #print("hi")
    return dbc.Table.from_dataframe(dataframe, striped=True, bordered=True, hover=True)

data,data_test = importdata() 
X, Y, X_train, X_test, y_train, y_test = splitdataset(data) 
clf_gini = train_using_gini(X_train, X_test, y_train,y_test) 
clf_entropy = tarin_using_entropy(X_train, X_test, y_train) 
app6 = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app6.config['suppress_callback_exceptions']=True
app7 = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app7.config['suppress_callback_exceptions']=True
app7.layout=html.Div([html.Div(id='page2',style={'textAlign': 'center','color': 'black'})])
app9 = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app9.config['suppress_callback_exceptions']=True
app9.layout=html.Div([html.Div(id='page3',style={'textAlign': 'center','color': 'black'})])
app6.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})
app6.layout = html.Div(style={'textAlign': 'center',"backgroundColor":"white"}
,children=[
        html.H1('STUDENT PERFORMANCE ANALYSIS SYSTEM USING MACHINE LEARNING',style={'text':'bold','textAlign': 'center','color':'red'}),
        html.Br(),html.Br(),
        html.Div(html.A('HOME PAGE', href='http://127.0.0.1:2020/', target='_self'),style={'textAlign': 'left'}),html.Br(),
        html.Div([html.Label('Select file :  '),dcc.Input(id='inp7',type='file')],style={'textAlign': 'center','color':'black'}),
        html.Br(),html.Br(),
        dcc.ConfirmDialog(id='confirm',message='only excel sheets with extension of .xls and .xlsx are allowed ',),
        html.Div(dbc.Button('predict', id='button3',color="primary", className="mr-1",n_clicks=0),style={'textAlign': 'center'}),
        html.Br(),html.Br(),
        html.Div(id='page1',style={'textAlign': 'center',"color":"black"})])
@app6.callback(dash.dependencies.Output('confirm', 'displayed'),
               [dash.dependencies.Input('inp7', 'value')])
def popup(inp7):
    if(inp7):
        #print(inp7)
        if((".xls" not in inp7) or(".xlsx" not in inp7)):
            return True
        else:
            return False

@app6.callback(dash.dependencies.Output('page1', 'children'),
              [dash.dependencies.Input('button3', 'n_clicks')],
                state=[State('inp7','value')])
def compute2(n_clicks,inp7):
       if(int(n_clicks)>0):
            #print(inp7)
            stulis=pd.read_excel(inp7)
            stulis10=stulis
            stulis1 = stulis.values[:, 2:4] 
            df3 = pd.DataFrame(stulis1)
            stuliresg=predictionres(df3,clf_gini)
            stulis10['GRADE']=stuliresg
            mycursor.execute(" DROP TABLE IF EXISTS `results`")
            stulis10.to_sql("results", engine, if_exists='replace')
            sub=stulis10["subject"].unique()
            f=stuliresg.count('F')
            #generate_graph(stulis10[['Rollnumber', 'subject','mid1','mid2','GRADES']])
            return html.Br(),html.Br(),html.Div(html.A(dbc.Button('Report',id='button4',color="primary", className="mr-1",n_clicks=0 ),href='http://127.0.0.1:2014/'),style={'textAlign': 'center'}),html.Br(), html.Div(html.A('Download Data',id='download-link',download="rawdata.csv",href="",target="_blank"),style={'textAlign': 'right'}),html.Div([generate_table(stulis10,)]),html.Br(),html.Div([generate_graph(stulis10,'count of each grade')]),html.Br(),html.Br(),html.Div([generate_piechart(stulis10,f)]),html.Br(),app7.layout

@app6.callback(
    dash.dependencies.Output('page2', 'children'),
     [dash.dependencies.Input('button3', 'n_clicks')],
                state=[State('inp7','value')])
def update_option(n_clicks,inp7):
    if(int(n_clicks)>0):
        #print("hihello")
        #print("file",inp7)
        stulis1=pd.read_excel(inp7)
        stulis110=stulis1
        stulis12 = stulis1.values[:, 2:4] 
        df31 = pd.DataFrame(stulis12)
        stuliresg1=predictionres(df31,clf_gini)
        stulis110['GRADE']=stuliresg1
        sub=stulis110["subject"].unique()
        #print(sub)
        opt=[{'label': i, 'value': i} for i in sub]
        #print(opt)
        return html.Label("Select Chart :"),html.Div(dcc.Dropdown(id='d4',options=opt,value=sub[0]),style={'vertical-align': 'middle','display': 'inline-block','height': '30px', 'width': '200px'}),app9.layout
@app6.callback(
    dash.dependencies.Output('page3', 'children'),
    [dash.dependencies.Input('d4','value'),dash.dependencies.Input('inp7', 'value')],
    )
def update_options(d4,inp7):
        #print('haha')
        p=str(d4)
        stulis=pd.read_excel(inp7)
        stulis10=stulis
        stulis1 = stulis.values[:, 2:4] 
        df3 = pd.DataFrame(stulis1)
        stuliresg=predictionres(df3,clf_gini)
        stulis10['GRADE']=stuliresg
        df4 = stulis10[stulis10['subject'].str.contains(p)]
        return html.Br(),html.Br(),html.Div([generate_graph(df4,d4)])
@app6.callback(
    dash.dependencies.Output('download-link', 'href'),
    [dash.dependencies.Input('button3', 'n_clicks')],
                state=[State('inp7','value')])
def update_download_link(n_clicks,inp7):
    if((n_clicks)>0):
        #print("hi")
        stulis1=pd.read_excel(inp7)
        stulis110=stulis1
        stulis12 = stulis1.values[:, 2:4] 
        df31 = pd.DataFrame(stulis12)
        stuliresg1=predictionres(df31,clf_gini)
        stulis110['GRADE']=stuliresg1
        dff = stulis110
        csv_string = dff.to_csv(index=False, encoding='utf-8')
        csv_string = "data:text/csv;charset=utf-8,%EF%BB%BF" 
        return csv_string
        
       

In [ ]:
if __name__ == '__main__':
    app6.run_server(port=2016)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2016/ (Press CTRL+C to quit)
C:\Users\coma\Anaconda3\lib\site-packages\dash\resources.py:75: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.ly/external-resources

127.0.0.1 - - [03/Apr/2020 00:54:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:51] "POST /_dash-update-component HTTP/1.1" 200 -
127

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\coma\Anaconda3\lib\site-packages\dash\dash.py", line 1461, in dispatch
    response.set_data(sel

127.0.0.1 - - [03/Apr/2020 01:03:04] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\coma\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\coma\Anaconda3\lib\site-packages\dash\dash.py", line 1461, in dispatch
    response.set_data(sel

127.0.0.1 - - [03/Apr/2020 01:03:07] "POST /_dash-update-component HTTP/1.1" 500 -
